<a href="https://colab.research.google.com/github/Pruthviraj141/RAG-Based-Models-/blob/main/Copy_of_ZERO_Cost_RAG_model_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q \
  langchain \
  langchain-core \
  langchain-community \
  langchain-chroma \
  langchain-openai \
  chromadb \
  pypdf


In [2]:
import os
import warnings
warnings.filterwarnings("ignore")

# 🔑 OpenRouter API Key (Free of costtttt)
os.environ["OPENAI_API_KEY"] = "My Dear Friend Use Your Own :D :D "

# 🔒 Disable LangSmith completely (IMPORTANT)
os.environ["LANGCHAIN_TRACING_V2"] = "false"


In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="google/gemma-3-27b-it:free",
    base_url="https://openrouter.ai/api/v1",
    temperature=0.2,
    default_headers={
        "HTTP-Referer": "https://your-site.com",   # optional
        "X-Title": "Hybrid PDF RAG",               # optional
    },
)


In [4]:

from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)


/tmp/ipython-input-4063482188.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [5]:
from google.colab import files
files.upload()


{}

In [6]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/pdf.pdf")
documents = loader.load()


In [15]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,          # smaller chunks
    chunk_overlap=80,        # less overlap
    separators=["\n\n", "\n", ".", " "],
)

splits = text_splitter.split_documents(documents)


In [16]:
import re

for doc in splits:
    if re.search(r"\b\d{10}\b", doc.page_content):
        doc.metadata["section"] = "contact"
    else:
        doc.metadata["section"] = "content"


In [19]:
import shutil
import os

CHROMA_PATH = "/content/chroma_pdf"

if os.path.exists(CHROMA_PATH):
    shutil.rmtree(CHROMA_PATH)

os.makedirs(CHROMA_PATH, exist_ok=True)


In [21]:
from langchain_chroma import Chroma


vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory="/content/chroma_pdf",
    collection_name="pdf_rag"
)


In [29]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
"""
You are a helpful, intelligent chatbot that uses a PDF as a primary knowledge source
but can also behave like a normal conversational assistant.

You must first classify the user's input into ONE of these categories:
A) PDF-related question
B) General knowledge question
C) Casual conversation / greeting / personal message

Follow these rules strictly:

--- CASE A: PDF-related question ---
If the question is about the document, its content, events, details, rules, contacts, or summary:
1. Check the PDF context.
2. If relevant information exists in the PDF:
   - Answer ONLY using the PDF content.
3. If the information does NOT exist in the PDF:
   - Say exactly:
     "Not found in the PDF.
      Using general knowledge, the answer is:"
   - Then answer using your general knowledge.

--- CASE B: General knowledge question ---
If the question is NOT about the PDF and is factual or informational:
- Answer using your general knowledge.
- Do NOT mention the PDF.

--- CASE C: Casual conversation ---
If the input is a greeting, introduction, opinion, or casual chat (for example:
"Hi", "My name is...", "How are you?", "I am from VIIT Pune"):
- Respond naturally like a friendly chatbot.
- Do NOT mention the PDF.
- Keep the tone polite and conversational.

General rules:
- Do NOT guess PDF information.
- Do NOT mix PDF facts with general knowledge.
- Keep responses concise and clear.
- Do NOT repeat previous answers.

PDF Context:
{context}

User Input:
{input}
"""
)


In [30]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
    # Explicit marker when retrieval is weak or empty
    if not docs or len(docs) == 0:
        return "NO_RELEVANT_PDF_CONTEXT"
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {
        "context": retriever | format_docs,
        "input": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)


In [31]:
rag_chain.invoke("what is document about ")


'A) PDF-related question\n\nThe document is about Impetus and Concepts (InC), the flagship international-level technical event organized by PICT, Pune. It is held in the third week of March 2025 and aims to help students turn ideas into impactful solutions and products.'

In [26]:
rag_chain.invoke("Provide me the contact Number of more details real quick")


'\nGauri Desale – 8669178247\nOmkar Deshmukh – 775683570\nAnushree Gattani – 8484024100.\n\n'

In [13]:
rag_chain.invoke("Who is Current President of Zimbabwe ")


'\nNot found in the PDF.\nUsing general knowledge, the answer is:\nThe current president of Zimbabwe is Emmerson Mnangagwa. He assumed office in November 2017, following the resignation of Robert Mugabe. Mnangagwa is a member of the ZANU-PF party, which has been the ruling party in Zimbabwe since its independence. His presidency has been marked by economic challenges and political reforms. He was re-elected in August 2023.'

In [32]:
rag_chain.invoke("HI my name is pruthviraj and im from viit pune ")


"\nHi Pruthviraj, nice to meet you! It's good to know you're from VIIT Pune. I'm here to help if you have any questions."